# Create Patches

In [ ]:
source_dir = os.path.abspath("/kaggle/input/fungal-10x/")
patch_dir = os.path.abspath('/kaggle/working/patches/')
patch_size = 256

In [ ]:
import os
import numpy as np
from PIL import Image
from itertools import product
import matplotlib.pyplot as plt

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/fungal-10x'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def tile(filename, dir_in, dir_out, d):
    if not os.path.isdir(dir_out):
        os.mkdir(dir_out)

    name, ext = os.path.splitext(filename)
    img = Image.open(os.path.join(dir_in, filename))
    w, h = img.size

    grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
    for i, j in grid:
        box = (j, i, j+d, i+d)
        i /= 256
        j /= 256
        out = os.path.join(dir_out, f'{name}_{int(i)}_{int(j)}{ext}')
        img.crop(box).save(out)


if not os.path.isdir(patch_dir):
    os.mkdir(patch_dir)

for filename in os.listdir(source_dir):
    name, ext = os.path.splitext(filename)
    output_patches_dir = os.path.join(patch_dir, name)
    
    print("Patching", filename)
    tile(filename, source_dir, output_patches_dir, patch_size)

# Feature Extraction

In [ ]:
patch_dir = os.path.abspath('/kaggle/working/patches/')
feat_dir = os.path.abspath('/kaggle/working/features/')

In [ ]:
import os
import yaml
import argparse

# import h5py
import cv2
import numpy as np
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.utils import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.callbacks import ModelCheckpoint

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# Create feat_dir if not exists.
if not os.path.isdir(feat_dir):
    os.mkdir(feat_dir)

# Loading ResNet50 wit imagenet weights, include_top means that we loading model without last fully connected layers
model = ResNet50(weights = 'imagenet', include_top = False)

# patch_folders = [os.path.join(patch_dir, folder) for folder in sorted(os.listdir(patch_dir))]
# patches_per_image = len(os.listdir(patch_folders[0]))
# print(patches_per_image)

# Create dataset from the image patches
for folder in sorted(os.listdir(patch_dir)):
    filename = str(folder).split("/")[-1]
    filePath = os.path.join(feat_dir, filename)
    # Run only if file doesn't already exist
    if os.path.exists(filePath):
        print("Skipping File:", filename)
        continue
    print("Running on File:", filename)

    features = []
    patch_folder = os.path.join(patch_dir, folder)
    for patch_file in sorted(os.listdir(patch_folder)):
        img_path = os.path.join(patch_folder, patch_file)

        # Get coord in [x, y] format
        coord = img_path.split("/")
        coord = coord[-1]
        coord = coord.split(".")[-2]
        coord = coord.split("_")
        coord = [int(coord[-2])/256, int(coord[-1])/256]

        # Read image
        orig = cv2.imread(img_path)

        # Convert image to RGB from BGR (another way is to use "image = image[:, :, ::-1]" code)
        orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

        # Resize image to 224x224 size
        image = cv2.resize(orig, (224, 224)).reshape(-1, 224, 224, 3)

        # We need to preprocess imageto fulfill ResNet50 requirements
        image = preprocess_input(image)

        # Extracting our features
        feature = model.predict(image)

        # Group the features
        features.append(feature)
    np.save(filePath, features)

# Dataset creator

In [ ]:
filename = os.path.join('/kaggle/working/', 'fungal_vs_nonfungal.csv')
patch_dir = os.path.abspath('/kaggle/working/patches/')
feat_dir = os.path.abspath('/kaggle/working/features/')
annotated_dir = os.path.abspath('/kaggle/input/fungal-10x-annot/')

In [ ]:
with open(filename, 'w') as file:
    file.write('case_id,slide_id,label' + '\n')

    patch_folders = [os.path.join(patch_dir, folder) for folder in sorted(os.listdir(patch_dir))]

    for i, name in enumerate(patch_folders):
        name = name.split("/")[-1]
        if name != feat_dir:
            if name[0] == "F":
                f_nf = "fungal"
            elif name[0] == "N":
                f_nf = "nonfungal"
                annotated = True
            else:
                f_nf = "unclassified"

            line = 'case_' + str(i) + ',' + name + ',' + f_nf
            file.write('{}\n'.format(line))


# Create Splits

In [24]:
label_frac = 1.0
seed = 1
k = 5
val_frac = 0.15
test_frac = 0.15
annot_frac = 0.4
annot_positive_frac = 1

dataset_csv_file = os.path.join('/kaggle/working/', 'fungal_vs_nonfungal.csv')

In [25]:
import os
import yaml
import random
import argparse
import numpy as np

In [26]:
from __future__ import print_function, division
import os
import torch
import numpy as np
import pandas as pd
import math
import re
import pdb
import pickle
import random
from scipy import stats

from torch.utils.data import Dataset
import h5py

In [27]:
def save_pkl(filename, save_object):
    writer = open(filename,'wb')
    pickle.dump(save_object, writer)
    writer.close()

def load_pkl(filename):
    loader = open(filename,'rb')
    file = pickle.load(loader)
    loader.close()
    return file

In [28]:
def generate_split(cls_ids, val_num, test_num, samples, n_splits = 5,
    seed = 7, label_frac = 1.0, custom_test_ids = None):
    indices = np.arange(samples).astype(int)

    if custom_test_ids is not None:
        indices = np.setdiff1d(indices, custom_test_ids)

    np.random.seed(seed)
    for i in range(n_splits):
        all_val_ids = []
        all_test_ids = []
        sampled_train_ids = []

        if custom_test_ids is not None: # pre-built test split, do not need to sample
            all_test_ids.extend(custom_test_ids)

        for c in range(len(val_num)):
            possible_indices = np.intersect1d(cls_ids[c], indices) #all indices of this class
            val_ids = np.random.choice(possible_indices, val_num[c], replace = False) # validation ids

            remaining_ids = np.setdiff1d(possible_indices, val_ids) #indices of this class left after validation
            all_val_ids.extend(val_ids)

            if custom_test_ids is None: # sample test split

                test_ids = np.random.choice(remaining_ids, test_num[c], replace = False)
                remaining_ids = np.setdiff1d(remaining_ids, test_ids)
                all_test_ids.extend(test_ids)

            if label_frac == 1:
                sampled_train_ids.extend(remaining_ids)

            else:
                sample_num  = math.ceil(len(remaining_ids) * label_frac)
                slice_ids = np.arange(sample_num)
                sampled_train_ids.extend(remaining_ids[slice_ids])

        yield sampled_train_ids, all_val_ids, all_test_ids


def nth(iterator, n, default=None):
    if n is None:
        return collections.deque(iterator, maxlen=0)
    else:
        return next(islice(iterator,n, None), default)

In [31]:
def save_splits(split_datasets, column_keys, filename, annot_frac=None, annot_positive_frac=None, boolean_style=False, annot_create=True):
    print(split_datasets)
    splits = [split_datasets[i].slide_data['slide_id'] for i in range(len(split_datasets))]

    if annot_create:
        # Add annot column # Only for 2 classes
        train_set = split_datasets[0]
        train_set_list = []
        annot_set = []
        positive_list = []
        negative_list = []

        for ids in train_set.slide_cls_ids[0]:
            negative_list.append(str(train_set.slide_data['slide_id'][ids]))

        for ids in train_set.slide_cls_ids[1]:
            positive_list.append(str(train_set.slide_data['slide_id'][ids]))

        train_set_list.extend(negative_list)
        train_set_list.extend(positive_list)

        train_set_annot = np.round(len(train_set_list) * annot_frac)
        neg_annot_num = np.round(train_set_annot * (1-annot_positive_frac)).astype(int)
        pos_annot_num = np.round(train_set_annot * annot_positive_frac).astype(int)

        neg_annot_set = random.sample(negative_list, neg_annot_num)
        pos_annot_set = random.sample(positive_list, pos_annot_num)

        annot_set.extend(neg_annot_set)
        annot_set.extend(pos_annot_set)

    #     print("annot_set", annot_set)

        true_annot_set = [False]*len(train_set_list)
        for idx in range(len(true_annot_set)):
            if train_set_list[idx] in annot_set:
                true_annot_set[idx] = True
    #     print("true_annot_set", true_annot_set)
        true_annot_set = pd.DataFrame(true_annot_set)
    #     print("splits", splits)
    #     print("true_annot_set", true_annot_set)
        splits.insert(1, true_annot_set)

    if not boolean_style:
        df = pd.concat(splits, ignore_index=True, axis=1)
        df.columns = column_keys
    else:
        df = pd.concat(splits, ignore_index = True, axis=0)
        index = df.values.tolist()
        one_hot = np.eye(len(split_datasets)).astype(bool)
        bool_array = np.repeat(one_hot, [len(dset) for dset in split_datasets], axis=0)
        df = pd.DataFrame(bool_array, index=index, columns = ['train', 'annot', 'val', 'test'])

    print(split_datasets[0].slide_data)
    df.to_csv(filename)
    print()

class Generic_WSI_Classification_Dataset(Dataset):
    def __init__(self,
        csv_path = 'dataset_csv/ccrcc_clean.csv',
        shuffle = False,
        seed = 7,
        print_info = True,
        label_dict = {},
        filter_dict = {},
        ignore=[],
        patient_strat=False,
        label_col = None,
        patient_voting = 'max',
        results_dir = None
        ):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            shuffle (boolean): Whether to shuffle
            seed (int): random seed for shuffling the data
            print_info (boolean): Whether to print a summary of the dataset
            label_dict (dict): Dictionary with key, value pairs for converting str labels to int
            ignore (list): List containing class labels to ignore
        """
        self.label_dict = label_dict
        self.num_classes = len(set(self.label_dict.values()))
        self.seed = seed
        self.print_info = print_info
        self.patient_strat = patient_strat
        self.train_ids, self.val_ids, self.test_ids  = (None, None, None)
        self.data_dir = None
        self.annot_dir = None
        if not label_col:
            label_col = 'label'
        self.label_col = label_col

        slide_data = pd.read_csv(csv_path)
        slide_data = self.filter_df(slide_data, filter_dict)
        slide_data = self.df_prep(slide_data, self.label_dict, ignore, self.label_col)
        print(slide_data)

        ###shuffle data
        if shuffle:
            np.random.seed(seed)
            np.random.shuffle(slide_data)

        self.slide_data = slide_data
        if results_dir:
            slide_data.to_csv(os.path.join(results_dir, 'dataset_csv.csv'))

        self.patient_data_prep(patient_voting)
        self.cls_ids_prep()

        if print_info:
            self.summarize()

    def cls_ids_prep(self):
        # store ids corresponding each class at the patient or case level
        self.patient_cls_ids = [[] for i in range(self.num_classes)]
        for i in range(self.num_classes):
            self.patient_cls_ids[i] = np.where(self.patient_data['label'] == i)[0]

        # store ids corresponding each class at the slide level
        self.slide_cls_ids = [[] for i in range(self.num_classes)]
        for i in range(self.num_classes):
            self.slide_cls_ids[i] = np.where(self.slide_data['label'] == i)[0]

    def patient_data_prep(self, patient_voting='max'):
        patients = np.unique(np.array(self.slide_data['case_id'])) # get unique patients
        patient_labels = []

        for p in patients:
            locations = self.slide_data[self.slide_data['case_id'] == p].index.tolist()
            assert len(locations) > 0
            label = self.slide_data['label'][locations].values
            if patient_voting == 'max':
                label = label.max() # get patient label (MIL convention)
            elif patient_voting == 'maj':
                label = stats.mode(label)[0]
            else:
                raise NotImplementedError
            patient_labels.append(label)

        self.patient_data = {'case_id':patients, 'label':np.array(patient_labels)}

    @staticmethod
    def df_prep(data, label_dict, ignore, label_col):
        if label_col != 'label':
            data['label'] = data[label_col].copy()

        mask = data['label'].isin(ignore)
        data = data[~mask]
        data.reset_index(drop=True, inplace=True)
        for i in data.index:
            key = data.loc[i, 'label']
            data.at[i, 'label'] = label_dict[key]

        return data

    def filter_df(self, df, filter_dict={}):
        if len(filter_dict) > 0:
            filter_mask = np.full(len(df), True, bool)
            # assert 'label' not in filter_dict.keys()
            for key, val in filter_dict.items():
                mask = df[key].isin(val)
                filter_mask = np.logical_and(filter_mask, mask)
            df = df[filter_mask]
        return df

    def __len__(self):
        if self.patient_strat:
            return len(self.patient_data['case_id'])

        else:
            return len(self.slide_data)

    def summarize(self):
        print("label column: {}".format(self.label_col))
        print("label dictionary: {}".format(self.label_dict))
        print("number of classes: {}".format(self.num_classes))
        print("slide-level counts: ", '\n', self.slide_data['label'].value_counts(sort = False))
        for i in range(self.num_classes):
            print('Patient-LVL; Number of samples registered in class %d: %d' % (i, self.patient_cls_ids[i].shape[0]))
            print('Slide-LVL; Number of samples registered in class %d: %d' % (i, self.slide_cls_ids[i].shape[0]))

    def create_splits(self, k = 3, val_num = (25, 25), test_num = (40, 40), label_frac = 1.0, custom_test_ids = None):
        settings = {
                    'n_splits' : k,
                    'val_num' : val_num,
                    'test_num': test_num,
                    'label_frac': label_frac,
                    'seed': self.seed,
                    'custom_test_ids': custom_test_ids
                    }

        if self.patient_strat:
            settings.update({'cls_ids' : self.patient_cls_ids, 'samples': len(self.patient_data['case_id'])})
        else:
            settings.update({'cls_ids' : self.slide_cls_ids, 'samples': len(self.slide_data)})

        self.split_gen = generate_split(**settings)

    def set_splits(self,start_from=None):
        if start_from:
            ids = nth(self.split_gen, start_from)

        else:
            ids = next(self.split_gen)

        if self.patient_strat:
            slide_ids = [[] for i in range(len(ids))]

            for split in range(len(ids)):
                for idx in ids[split]:
                    case_id = self.patient_data['case_id'][idx]
                    slide_indices = self.slide_data[self.slide_data['case_id'] == case_id].index.tolist()
                    slide_ids[split].extend(slide_indices)

            self.train_ids, self.val_ids, self.test_ids = slide_ids[0], slide_ids[1], slide_ids[2]

        else:
            self.train_ids, self.val_ids, self.test_ids = ids

    def get_split_from_df(self, all_splits, split_key='train'):
        split = all_splits[split_key]
        split = split.dropna().reset_index(drop=True)

        if len(split) > 0:
            mask = self.slide_data['slide_id'].isin(split.tolist())
            df_slice = self.slide_data[mask].reset_index(drop=True)
            split = Generic_Split(df_slice, data_dir=self.data_dir, annot_dir=self.annot_dir, num_classes=self.num_classes)
        else:
            split = None

        return split

    def get_merged_split_from_df(self, all_splits, split_keys=['train']):
        merged_split = []
        for split_key in split_keys:
            split = all_splits[split_key]
            split = split.dropna().reset_index(drop=True).tolist()
            merged_split.extend(split)

        if len(split) > 0:
            mask = self.slide_data['slide_id'].isin(merged_split)
            df_slice = self.slide_data[mask].reset_index(drop=True)
            split = Generic_Split(df_slice, data_dir=self.data_dir, num_classes=self.num_classes)
        else:
            split = None

        return split

    def get_overlap_split_from_df(self, all_splits, split_keys=['train', 'annot']):
        train_split = all_splits['train']
        annot_split = all_splits['annot']

        if len(train_split) > 0:
            mask = self.slide_data['slide_id'].isin(train_split)
            df_slice = self.slide_data[mask].reset_index(drop=True)

            mask = train_split.isin(df_slice['slide_id'].tolist())
            df_slice['annot'] = annot_split[mask]

            split = Generic_Split(df_slice, data_dir=self.data_dir, annot_dir=self.annot_dir, num_classes=self.num_classes)
        else:
            split = None

        return split


    def return_splits(self, from_id=True, csv_path=None):


        if from_id:
            if len(self.train_ids) > 0:
                train_data = self.slide_data.loc[self.train_ids].reset_index(drop=True)
                train_split = Generic_Split(train_data, annot_dir=self.annot_dir, data_dir=self.data_dir, num_classes=self.num_classes)

            else:
                train_split = None

            if len(self.val_ids) > 0:
                val_data = self.slide_data.loc[self.val_ids].reset_index(drop=True)
                val_split = Generic_Split(val_data, data_dir=self.data_dir, num_classes=self.num_classes)

            else:
                val_split = None

            if len(self.test_ids) > 0:
                test_data = self.slide_data.loc[self.test_ids].reset_index(drop=True)
                test_split = Generic_Split(test_data, data_dir=self.data_dir, num_classes=self.num_classes)

            else:
                test_split = None


        else:
            assert csv_path
            all_splits = pd.read_csv(csv_path, dtype=self.slide_data['slide_id'].dtype)  # Without "dtype=self.slide_data['slide_id'].dtype", read_csv() will convert all-number columns to a numerical type. Even if we convert numerical columns back to objects later, we may lose zero-padding in the process; the columns must be correctly read in from the get-go. When we compare the individual train/val/test columns to self.slide_data['slide_id'] in the get_split_from_df() method, we cannot compare objects (strings) to numbers or even to incorrectly zero-padded objects/strings. An example of this breaking is shown in https://github.com/andrew-weisman/clam_analysis/tree/main/datatype_comparison_bug-2021-12-01.
            train_split = self.get_overlap_split_from_df(all_splits, ['train', 'annot'])
            val_split = self.get_overlap_split_from_df(all_splits, ['val', 'annot'])
            test_split = self.get_overlap_split_from_df(all_splits, ['test', 'annot'])
            # train_split = self.get_split_from_df(all_splits, 'train')
            # val_split = self.get_split_from_df(all_splits, 'val')
            # test_split = self.get_split_from_df(all_splits, 'test')

        return train_split, val_split, test_split

    def get_list(self, ids):
        return self.slide_data['slide_id'][ids]

    def getlabel(self, ids):
        return self.slide_data['label'][ids]

    def __getitem__(self, idx):
        return None

    def test_split_gen(self, return_descriptor=False):

        if return_descriptor:
            index = [list(self.label_dict.keys())[list(self.label_dict.values()).index(i)] for i in range(self.num_classes)]
            columns = ['train', 'val', 'test']
            df = pd.DataFrame(np.full((len(index), len(columns)), 0, dtype=np.int32), index= index,
                            columns= columns)

        count = len(self.train_ids)
        print('\nnumber of training samples: {}'.format(count))
        labels = self.getlabel(self.train_ids)
        unique, counts = np.unique(labels, return_counts=True)
        for u in range(len(unique)):
            print('number of samples in cls {}: {}'.format(unique[u], counts[u]))
            if return_descriptor:
                df.loc[index[u], 'train'] = counts[u]

        count = len(self.val_ids)
        print('\nnumber of val samples: {}'.format(count))
        labels = self.getlabel(self.val_ids)
        unique, counts = np.unique(labels, return_counts=True)
        for u in range(len(unique)):
            print('number of samples in cls {}: {}'.format(unique[u], counts[u]))
            if return_descriptor:
                df.loc[index[u], 'val'] = counts[u]

        count = len(self.test_ids)
        print('\nnumber of test samples: {}'.format(count))
        labels = self.getlabel(self.test_ids)
        unique, counts = np.unique(labels, return_counts=True)
        for u in range(len(unique)):
            print('number of samples in cls {}: {}'.format(unique[u], counts[u]))
            if return_descriptor:
                df.loc[index[u], 'test'] = counts[u]

        assert len(np.intersect1d(self.train_ids, self.test_ids)) == 0
        assert len(np.intersect1d(self.train_ids, self.val_ids)) == 0
        assert len(np.intersect1d(self.val_ids, self.test_ids)) == 0

        if return_descriptor:
            return df

    def save_split(self, filename):
        train_split = self.get_list(self.train_ids)
        val_split = self.get_list(self.val_ids)
        test_split = self.get_list(self.test_ids)
        df_tr = pd.DataFrame({'train': train_split})
        df_v = pd.DataFrame({'val': val_split})
        df_t = pd.DataFrame({'test': test_split})
        df = pd.concat([df_tr, df_v, df_t], axis=1)
        df.to_csv(filename, index = False)


class Generic_MIL_Dataset(Generic_WSI_Classification_Dataset):
    def __init__(self,
        data_dir,
        annot_dir=None,
        **kwargs):

        super(Generic_MIL_Dataset, self).__init__(**kwargs)
        self.data_dir = data_dir
        self.annot_dir = annot_dir
        self.use_h5 = False

    def load_from_h5(self, toggle):
        self.use_h5 = toggle

    def __getitem__(self, idx):
        slide_id = self.slide_data['slide_id'][idx]
        label = self.slide_data['label'][idx]
        if self.slide_data['annot'][idx]:
            bool_annot = bool(self.slide_data['annot'][idx])
            if label == 1:
                patch_annot_path = os.path.join(self.annot_dir, slide_id, slide_id+'.pkl')
                patch_annot = load_pkl(patch_annot_path)
                patch_annot = patch_annot['bin_scores']
            elif label == 0:
                patch_annot = [False]*24

        if type(self.data_dir) == dict:
            source = self.slide_data['source'][idx]
            data_dir = self.data_dir[source]
        else:
            data_dir = self.data_dir

        if not self.use_h5:
            if self.data_dir:
                full_path = os.path.join(data_dir, '{}.pt'.format(slide_id))
                features = torch.load(full_path)
                return features, label, bool_annot, patch_annot
                # return features, label

            else:
                # if bool_annot:
                #     return slide_id, label, bool_annot, patch_annot
                # else:
                #     return slide_id, label, None, None
                return slide_id, label

        else:
            full_path = os.path.join(data_dir,'h5_files','{}.h5'.format(slide_id))
            with h5py.File(full_path,'r') as hdf5_file:
                features = hdf5_file['features'][:]
                coords = hdf5_file['coords'][:]

            features = torch.from_numpy(features)
            return features, label, coords


class Generic_Split(Generic_MIL_Dataset):
    def __init__(self, slide_data, annot_dir=None, data_dir=None, num_classes=2):
        self.use_h5 = False
        self.slide_data = slide_data
        self.data_dir = data_dir
        self.annot_dir = annot_dir
        self.num_classes = num_classes
        self.slide_cls_ids = [[] for i in range(self.num_classes)]
        for i in range(self.num_classes):
            self.slide_cls_ids[i] = np.where(self.slide_data['label'] == i)[0]

    def __len__(self):
        return len(self.slide_data)

In [32]:
random.seed(seed)

# task_1_fungal_vs_nonfungal
n_classes=2
dataset = Generic_WSI_Classification_Dataset(csv_path = dataset_csv_file,
                        shuffle = False,
                        seed = seed,
                        print_info = True,
                        label_dict = {'nonfungal':0, 'fungal':1},
                        patient_strat=True,
                        ignore=[])

num_slides_cls = np.array([len(cls_ids) for cls_ids in dataset.patient_cls_ids])
val_num = np.round(num_slides_cls * val_frac).astype(int)
test_num = np.round(num_slides_cls * test_frac).astype(int)

if label_frac > 0:
    label_fracs = [label_frac]
else:
    label_fracs = [0.1, 0.25, 0.5, 0.75, 1.0]

for lf in label_fracs:
    split_dir = 'splits/fungal_vs_nonfungal' + '_{}'.format(int(lf * 100))
    os.makedirs(split_dir, exist_ok=True)
    dataset.create_splits(k = k, val_num = val_num, test_num = test_num, label_frac=lf)
    for i in range(k):
        dataset.set_splits()
        descriptor_df = dataset.test_split_gen(return_descriptor=True)
        splits = dataset.return_splits(from_id=True)

        save_splits(splits, ['train', 'annot', 'val', 'test'], os.path.join(split_dir, 'splits_{}.csv'.format(i)), annot_frac=annot_frac, annot_positive_frac=annot_positive_frac)
        # save_splits(splits, ['train', 'annot', 'val', 'test'], os.path.join(split_dir, 'splits_{}_bool.csv'.format(i)), boolean_style=True)
        # descriptor_df.to_csv(os.path.join(split_dir, 'splits_{}_descriptor.csv'.format(i)))

      case_id  slide_id label
0      case_0   F005a02     1
1      case_1   F006a01     1
2      case_2   F006a02     1
3      case_3   F006a03     1
4      case_4   F006a04     1
..        ...       ...   ...
418  case_418  N012a017     0
419  case_419  N012a018     0
420  case_420  N012a019     0
421  case_421  N012a020     0
422  case_422  N017a001     0

[423 rows x 3 columns]
label column: label
label dictionary: {'nonfungal': 0, 'fungal': 1}
number of classes: 2
slide-level counts:  
 1    208
0    215
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 215
Slide-LVL; Number of samples registered in class 0: 215
Patient-LVL; Number of samples registered in class 1: 208
Slide-LVL; Number of samples registered in class 1: 208

number of training samples: 297
number of samples in cls 0: 151
number of samples in cls 1: 146

number of val samples: 63
number of samples in cls 0: 32
number of samples in cls 1: 31

number of test samples: 63
number of samples 

In [36]:
# View the splits
# !cat splits/fungal_vs_nonfungal_100/splits_0.csv